# Usage Overview
The following is example usage for `harmony-py`. 


In [ ]:

# Under this proposed interface harmony-py would have the concept of three (3) entities: a Request, a Client, and a Job:
# - The Request contains request parameters
# - The Client represents the ability to perform authenticated HTTP with the Harmony endpoint
# - The Job represents what Harmony is working on as well as retrieving the finished work results. It's referenced via 'job_id' and used by the Client.

# Individual parameters are validated when set.
# Not all keyword args need be supplied at once. Also, parameters may be replaced.
request = Request(
    collection=Collection(id='C1940468263-POCLOUD'),
    spatial={'ll': (40, -107),
             'ur': (42, -105)}
    temporal={'start': datetime.date(2020, 6, 1),
              'stop': datetime.date(2020, 6, 30)},
    format=Format.ZARR
)

# Authentication is stored in a client object for subsequent server interaction.
client = Client(Authentication())

# Validation may be performed prior to job processing; uses Harmony server-side checking.
client.validate(request)

# Starts job processing; async by default.
job_id = client.submit(request, verbose=True, async=True)

# Optional
client.status(job_id)

# Optional
client.cancel(job_id)

# Retrieve results in-region; returns a generator
urls = client.result_urls(job_id, region=Region.US_WEST_2)
# - or -
# Download files to a local directory
client.download(job_id, region=Region.US_WEST_2, directory='./research', overwrite=True)


# Further Examples

## Authentication

In [ ]:
# Authentication options:
#   .) specify `username` and `password`
#   .) specify `username` and receive password prompt
#   .) specify .netrc
#   .) read .netrc in default location
#   .) read .env file
#   .) read ENV vars    

auth = Authenticate(username='myusername', password='supersecret')
# or
auth = Authenticate(username='myusername')
# or
auth = Authenticate(netrc='/usr/local/etc/some/path/.netrc')

# or
auth = Authenticate()


## Determine Service Availability and Variables / Working with CMR

In [ ]:
# Notes from a previous meeting:
# extend CMR library to UMM-S; nice output
# when and where do we use Harmony's Capabilities documents?
# stick with Python data structures (dicts, lists, etc.)
# output of CMR should be acceptable input to Harmony python lib
# but also allow a user to submit strings as input for Harmony python lib
# understand UMM-Var response (coupled ot their metadata format)

# More Notes: We may want to contribute to the CMR python library in order to make feeding data into the Harmony python library easier than what's shown here.

import re

# Import CMR's python library
import cmr.search.collection as coll
import cmr.search.granule as gran

cmr_res = coll.search({'keyword':'MOD09*',
                       'archive_center': 'lp daac'})

# regex uses a negative look-around assertion
brief = [[r['meta']['concept-id'],
          r['umm']['ShortName'],
          r['meta']['native-id']] for r in cmr_res if re.search('^((?!mmt_collection_).)*$', r['meta']['native-id'])]
[print(b) for b in brief]
# ['C193529903-LPDAAC_ECS', 'MOD09GQ', 'MODIS/Terra Surface Reflectance Daily L2G Global 250m SIN Grid V006']
# ['C193529902-LPDAAC_ECS', 'MOD09GA', 'MODIS/Terra Surface Reflectance Daily L2G Global 1km and 500m SIN Grid V006']
# ['C193529899-LPDAAC_ECS', 'MOD09A1', 'MODIS/Terra Surface Reflectance 8-Day L3 Global 500m SIN Grid V006']
# ['C193529944-LPDAAC_ECS', 'MOD09Q1', 'MODIS/Terra Surface Reflectance 8-Day L3 Global 250m SIN Grid V006']
# ['C193529901-LPDAAC_ECS', 'MOD09CMG', 'MODIS/Terra Surface Reflectance Daily L3 Global 0.05Deg CMG V006']


####
# The CMR python library does not support variable browsing at this time.
####


# The output from CMR may be used as input to the Harmony python library
req = Request(
    collection=cmr_res[0],
    spatial=Bbox(lat=(40, 42), lon=(-107, -105)),  # could accept bbox, shapely, geojson (polygon)
    temporal=Temporal(start=datetime.date(2020, 6, 1), stop=datetime.date(2020, 6, 30)),
    format=Format.ZARR
)


## Async vs. Sync Request Submit()

In [ ]:
# Sync Request
res = Request.submit(req, auth, sync=True)

# Async request; default behavior
res = Request.submit(req, auth)

# Async usage: Poll Harmony status page and display progress updates.
res.update()

# Async usage: Cancel an ongoing job (Result)
res.cancel()


## Retrieve Results in Cloud: In / Out of Region; Internet Retrieval

In [ ]:
# Notes:
# require the user to be explicit on style of retrieval: in-cloud/in-region vs. over internet
# if in same region, collect k/v pairs for parameters to boto3 constructor
# End-user _must_ specify region keyword argument.

import boto3
import requests

# Downloads files to a local directory. Easy method.
Response.download(res, region=Response.IN_REGION, directory='./research', overwrite=True)

# Downloads files to a local directory but skips files which already exist; Note: doesn't verify existing file size. Easy method.
Response.download(res, region=Response.IN_REGION, directory='./research', overwrite=False)

####

# In-Region; alternative to the above.
s3 = boto3.client('s3')
files = Response.files(res, region=Response.IN_REGION)
for f in files:
    # The parameters for each output file are the inputs to this boto method.
    s3.download_file(f.bucket_name, f.object_name, './research/' + f.filename)

# Out-of-Region; alternative to the above easy methods.
files = Response.files(res, region=Response.OUT_OF_REGION)
for f in files:
    r = requests.get(f.url, allow_redirects=True)
    open(f.filename, 'wb').write(r.content)


## Error Notification and Handling

Open for suggestions. We probably should raise exceptions as needed and output friendly messages via logging. The same logging will be used for async operation. Async is futures based so the GIL and cooperative multitasking will handle contention for logging output destinations. STDOUT will be the default logging target.

Notes from 2021/2/8 meeting:
- no Temporal class
  - use datetime objects but not strings; tuple is fine; allow None on either side
  - could also use a NamedTuple or @dataclass
  - possible python construct here
- for Bbox
  - don't provide just numbers of the form "12:34" (OGC standard)
  - follow CMR python library usage? Or send them a PR to align things with harmony-py
- remove req.action
- Request.validate(req)
  - throw an exception that contains the individual errors
  - 'validate' should exist in Harmony and be a function
  - perform HEAD request (or something that allows for responses in body) and send to Harmony; use server-side validation
- Request.submit() lift up
  - will need to get Harmony to have status page even on sync requests
- Response object could yield a sequence of e.g. URLs; watch for coroutine usage on resetting sequence
  - download() would continue to download files in the background
- instead of IN_REGION it should specify us-west-2 somehow
- everything requires auth for server communication; a 'client' or something should have auth as a part of it. Does not need to hold up short-term development.

- UMM-S and UMM-Var browsing via CMR python library
  - ... meeting ended here

